In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import matplotlib.pyplot as plt
import random
import sklearn
import spacy
nlp = spacy.load('de_core_news_md')
# import fasttext

import re
from bs4 import BeautifulSoup
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

In [27]:
from content_based_model import ContentBasedRecommender
from evaluator import ModelEvaluator

In [2]:
articles = pd.read_csv('./data/articles_clean.csv')
articles_lemma = pd.read_csv('./data/articles_lemmatized.csv')

## Wektoryzacja

In [6]:
stopwords_list = stopwords.words('german') # are there other languages in text

### Bag of words

In [7]:
bag_vectorizer = CountVectorizer(analyzer='word',
                     ngram_range=(1, 3), # bi - gram (?)
                     min_df=0.01,
                     max_df=0.7,
                     max_features=5000,
                     stop_words=stopwords_list)


### TFIDF

In [8]:

tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 3), # bi - gram (?)
                     min_df=0.01,
                     max_df=0.7,
                     max_features=5000,
                     stop_words=stopwords_list)


### Word2vec

In [9]:
# to do 

### FastText

In [10]:
# to do 
model = fasttext.load_model('model.bin')
vect = model.get_sentence_vector("some string") # 1 sentence
vect2 = [model.get_sentence_vector(el.replace('\n', '')) for el in text] # for text

NameError: name 'fasttext' is not defined

In [11]:
def vectorize(vectorizer, data): # tylko do tfidf/count (?)
    matrix  = vectorizer.fit_transform(data['content'])
    feature_names = vectorizer.get_feature_names()
    return matrix, feature_names

In [12]:
item_ids = articles['nzz_id'].tolist() # w lemma te same 

In [13]:
tfidf_matrix, tfidf_features = vectorize(tfidf_vectorizer, articles)
bag_matrix, bag_features = vectorize(bag_vectorizer, articles)


In [14]:
tfidf_matrix_lemma, tfidf_features_lemma = vectorize(tfidf_vectorizer, articles_lemma)
bag_matrix_lemma, bag_features_lemma = vectorize(bag_vectorizer, articles_lemma)

### User profiles (?)

In [15]:
readers = pd.read_csv('./../readers.csv')

In [16]:
readers['eventStrength'] = 1

In [17]:
# users_interactions_count_df = readers.groupby(['id', 'art_id']).size().groupby('id').size()
# print('# users: %d' % len(users_interactions_count_df))
# users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['id']]
# print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

In [18]:
interactions_train, interactions_test = train_test_split(readers,
                                   stratify=readers['id'], 
                                   test_size=0.20,
                                   random_state=123)

print('# interactions on Train set: %d' % len(interactions_train))
print('# interactions on Test set: %d' % len(interactions_test))

# interactions on Train set: 22284
# interactions on Test set: 5571


In [19]:
#Indexing by personId to speed up the searches during evaluation
interactions_total_ind = readers.set_index('id')
interactions_train_ind = interactions_train.set_index('id')
interactions_test_ind = interactions_test.set_index('id')

def get_items_interacted(person_id, interactions):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions.loc[person_id]['art_id']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [20]:
get_items_interacted(1,interactions_total_ind)

{'1.18331199', 'ld.1293110', 'ld.142559', 'ld.144819', 'ld.154103'}

In [21]:
# To model the user profile, we take all the item profiles the user has interacted and average them. The average is weighted by the interaction strength, in other words, the articles the user has interacted the most (eg. liked or commented) will have a higher strength in the final user profile.

def get_item_profile(item_id, matrix):
    idx = item_ids.index(item_id)
    item_profile = matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids,matrix):
    item_profiles_list = [get_item_profile(x,matrix) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df,matrix):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['art_id'],matrix)
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1) # czy potrzebne od tego momentu?
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm
    # return user_item_profiles

def build_users_profiles(matrix): 
    interactions_indexed_df = interactions_train[interactions_train['art_id'] \
                                                   .isin(articles['nzz_id'])].set_index('id')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df,matrix)
    return user_profiles

In [22]:
interactions_ind = interactions_train[interactions_train['art_id'] \
                                                   .isin(articles['nzz_id'])].set_index('id')
interactions_person = interactions_ind.loc[1]
get_item_profiles(interactions_person['art_id'], tfidf_matrix)

<4x2282 sparse matrix of type '<class 'numpy.float64'>'
	with 132 stored elements in Compressed Sparse Row format>

In [23]:
tfidf_profiles = build_users_profiles(tfidf_matrix)
bag_profiles = build_users_profiles(bag_matrix)
len(tfidf_profiles)

1000

In [24]:
tfidf_profiles_lemma = build_users_profiles(tfidf_matrix_lemma)
bag_profiles_lemma = build_users_profiles(tfidf_matrix_lemma)


In [25]:
myprofile = tfidf_profiles[1]
print(myprofile.shape)
pd.DataFrame(sorted(zip(tfidf_features, 
                        myprofile.flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

(1, 2282)


,token,relevance
0,insel,0.248814
1,papier,0.234971
2,frauen,0.227556
3,mal,0.216718
4,überall,0.210696
5,de,0.179554
6,leicht,0.177821
7,verkaufen,0.174356
8,zudem,0.162986
9,wasser,0.161277


## Ewaluator

In [31]:
# topn accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_total_ind)
        all_items = set(articles['nzz_id'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_ind.loc[person_id]
        if type(interacted_values_testset['art_id']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['art_id'])
        else:
            person_interacted_items_testset = set([(interacted_values_testset['art_id'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_ind), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=123)

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['art_id'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['art_id'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_ind.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df

In [32]:
content_based_recommender_model = ContentBasedRecommender(item_ids,articles, tfidf_profiles,tfidf_matrix)
model_evaluator = ModelEvaluator()  

In [33]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
999 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.10770059235325795, 'recall@10': 0.12457368515526836}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
358,2,2,10,0.2,0.2,907
721,1,1,10,0.1,0.1,938
510,2,2,10,0.2,0.2,64
86,2,2,10,0.2,0.2,887
484,2,2,10,0.2,0.2,397
472,2,2,10,0.2,0.2,218
259,3,3,10,0.3,0.3,273
264,0,0,10,0.0,0.0,803
797,2,2,10,0.2,0.2,865
276,2,2,10,0.2,0.2,443


In [97]:
content_based_recommender_model_2 = ContentBasedRecommender(articles, bag_profiles, bag_matrix)
model_evaluator = ModelEvaluator()

In [98]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model_2)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
999 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.09639203015616586, 'recall@10': 0.10482857655717107}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
358,2,3,10,0.2,0.3,907
721,1,1,10,0.1,0.1,938
510,1,1,10,0.1,0.1,64
86,1,1,10,0.1,0.1,887
484,1,2,10,0.1,0.2,397
472,1,3,10,0.1,0.3,218
259,1,1,10,0.1,0.1,273
264,0,0,10,0.0,0.0,803
797,1,2,10,0.1,0.2,865
276,2,2,10,0.2,0.2,443


In [136]:
content_based_recommender_model_3 = ContentBasedRecommender(articles_lemma, bag_profiles_lemma, bag_matrix_lemma)
model_evaluator = ModelEvaluator()

In [137]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model_3)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
999 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.10716208939149165, 'recall@10': 0.11541913480524144}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
358,2,2,10,0.2,0.2,907
721,1,1,10,0.1,0.1,938
510,2,2,10,0.2,0.2,64
86,1,1,10,0.1,0.1,887
484,2,2,10,0.2,0.2,397
472,1,3,10,0.1,0.3,218
259,3,3,10,0.3,0.3,273
264,0,0,10,0.0,0.0,803
797,0,3,10,0.0,0.3,865
276,1,1,10,0.1,0.1,443


In [138]:
content_based_recommender_model_4 = ContentBasedRecommender(articles_lemma, tfidf_profiles_lemma, tfidf_matrix_lemma)
model_evaluator = ModelEvaluator()

In [139]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model_4)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
999 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.11039310716208939, 'recall@10': 0.11990665948662718}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
358,2,2,10,0.2,0.2,907
721,1,1,10,0.1,0.1,938
510,2,2,10,0.2,0.2,64
86,1,1,10,0.1,0.1,887
484,2,2,10,0.2,0.2,397
472,2,3,10,0.2,0.3,218
259,3,3,10,0.3,0.3,273
264,0,0,10,0.0,0.0,803
797,1,2,10,0.1,0.2,865
276,1,1,10,0.1,0.1,443
